In [1]:
import polars as pl
import numpy as np
import os

In [4]:
data_dir = "/Users/maxwoolf/Downloads"

df = pl.read_parquet(
    os.path.join(data_dir, "movie_data_plus_embeds_all.parquet"),
).with_columns(
    title=pl.col("json").str.json_path_match(r"$.title"),
)

df

tconst,startYear,numVotes,averageRating,json,embeds,title
str,i64,i64,f64,str,"array[f32, 768]",str
"""tt1921149""",2014,23196,7.1,"""{""title"":""Trash"",""genres"":[""Ad…","[0.035435, 0.038249, … -0.043424]","""Trash"""
"""tt10944276""",2020,718,4.8,"""{""title"":""Death of a Telemarke…","[0.011852, 0.023681, … -0.014766]","""Death of a Telemarketer"""
"""tt0116163""",1996,5113,6.5,"""{""title"":""Ebola Syndrome"",""gen…","[0.014408, 0.011426, … -0.015651]","""Ebola Syndrome"""
"""tt13176330""",2024,615,5.0,"""{""title"":""Hitpig"",""genres"":[""A…","[0.020329, 0.077377, … -0.047044]","""Hitpig"""
"""tt6594070""",2018,73,4.7,"""{""title"":""The October Flowers""…","[-0.001671, 0.046787, … -0.037302]","""The October Flowers"""
…,…,…,…,…,…,…
"""tt0467110""",2007,23137,4.8,"""{""title"":""Underdog"",""genres"":[…","[0.04189, 0.008715, … -0.013782]","""Underdog"""
"""tt0139678""",1941,66,5.8,"""{""title"":""T\u00e4ysosuma"",""gen…","[0.025161, 0.060281, … -0.069796]","""Täysosuma"""
"""tt2071571""",2013,2398,4.1,"""{""title"":""Scarecrow"",""genres"":…","[-0.005015, 0.00519, … -0.039385]","""Scarecrow"""


In [6]:
embeds = df["embeds"].to_numpy()
embeds.shape

(236771, 768)

Test Star Wars


In [8]:
movie = df.filter(pl.col("tconst") == "tt0076759").rows(named="true")[0]
query_embed = np.array(movie["embeds"])
query_embed.shape

(768,)

In [9]:
cossims = query_embed @ embeds.T
cossims[:10]

array([0.56890133, 0.58723163, 0.55072288, 0.6853516 , 0.54252828,
       0.60906127, 0.58292649, 0.61072288, 0.64010188, 0.53257757])

In [21]:
# https://stackoverflow.com/a/6910672
top_idx = np.argsort(cossims)[-100:][::-1]
top_idx

array([206252, 206628, 104432, 177372,  88930,  37690,  39146, 219943,
       152254,  51136, 158526, 117392, 147588, 163276, 191297, 188044,
       231111, 181583, 190071, 234292, 113867, 233509, 113548,  46825,
       140475, 229203, 103314, 140033, 207626,  54017, 222820,  37589,
       195078, 108529,  57292, 132680,  73393, 171024,  92700, 183536,
       100663, 118154,   8739, 157522,  22373,  85719, 141190,  14653,
       122173,  33676,  38759,  70295,  27806,  70276, 197863,  63810,
       108001, 104241,  49830, 135463, 147786, 150316, 127640, 110705,
       110422, 199781,  80288,  84124,  10010,  16329,  37647,  11450,
        84368, 183368, 113580, 161952, 118667,  61498,  96166,  45385,
       146370,  16844, 147536, 214846, 182658, 232815,  81480, 167264,
        35235, 159966, 110833,  85098, 164804, 118929, 194214,  21305,
       158513, 231213,  59057, 201674])

In [24]:
top_cossims = cossims[top_idx]
top_cossims

array([0.99999997, 0.86571068, 0.82682951, 0.8263989 , 0.81977434,
       0.81941506, 0.81118548, 0.81045145, 0.80614216, 0.79548957,
       0.79470226, 0.775438  , 0.77109689, 0.77057338, 0.76956178,
       0.76827411, 0.76806791, 0.76665124, 0.76574274, 0.76474259,
       0.7615931 , 0.76038585, 0.75951698, 0.75853496, 0.75702035,
       0.75577689, 0.75485201, 0.75183444, 0.75121371, 0.75026687,
       0.74942665, 0.74814597, 0.74584653, 0.73951284, 0.7393803 ,
       0.73909254, 0.73737114, 0.73729253, 0.7365705 , 0.73437542,
       0.73314665, 0.73060532, 0.72742735, 0.72294677, 0.7225022 ,
       0.72042262, 0.71983925, 0.71527596, 0.71473636, 0.71472928,
       0.71275583, 0.71215041, 0.71201581, 0.70974159, 0.70831535,
       0.70731782, 0.70731532, 0.70568963, 0.70553747, 0.704865  ,
       0.70460576, 0.7045627 , 0.70438034, 0.70408787, 0.7039469 ,
       0.70365459, 0.7031081 , 0.70285143, 0.7026842 , 0.70259596,
       0.7022299 , 0.7017879 , 0.70137641, 0.70136008, 0.70128

In [26]:
df_related = pl.DataFrame(
    {"title": df[top_idx]["title"].to_list(), "cossim": top_cossims}
)
df_related

title,cossim
str,f64
"""Star Wars: Episode IV - A New …",1.0
"""Star Wars: Episode V - The Emp…",0.865711
"""Star Wars: Episode VI - Return…",0.82683
"""Star Wars: Episode VII - The F…",0.826399
"""Star Wars: Episode VIII - The …",0.819774
…,…
"""Magnus""",0.696506
"""Parallels""",0.696425
"""Knights""",0.696394


## Functionalize


In [35]:
def related_movies(movie_id):
    movie = df.filter(pl.col("tconst") == movie_id).rows(named="true")[0]
    query_embed = np.array(movie["embeds"])

    cossims = query_embed @ embeds.T
    top_idx = np.argsort(cossims)[-100:][::-1]
    top_cossims = cossims[top_idx]

    with pl.Config(tbl_rows=20):
        print(
            pl.DataFrame(
                {"title": df[top_idx]["title"].to_list(), "cossim": top_cossims}
            )
        )

In [ ]:
# Sonic the Hedgehog 3
related_movies("tt18259086")

shape: (100, 2)
┌────────────────────────────┬──────────┐
│ title                      ┆ cossim   │
│ ---                        ┆ ---      │
│ str                        ┆ f64      │
╞════════════════════════════╪══════════╡
│ Sonic the Hedgehog 3       ┆ 1.0      │
│ Sonic the Hedgehog         ┆ 0.949313 │
│ Sonic the Hedgehog 2       ┆ 0.925007 │
│ Sonic Impact               ┆ 0.798665 │
│ The Hedgehog               ┆ 0.795263 │
│ Hedgehogs                  ┆ 0.793229 │
│ Sonic Sea                  ┆ 0.792646 │
│ Sonic Rebuilt              ┆ 0.776375 │
│ Hedgehog                   ┆ 0.766838 │
│ The Hedgehogs              ┆ 0.766778 │
│ …                          ┆ …        │
│ Guido                      ┆ 0.710525 │
│ Jimmy                      ┆ 0.710522 │
│ Hop                        ┆ 0.710446 │
│ Echo Dr.                   ┆ 0.710361 │
│ Pearl                      ┆ 0.710284 │
│ Heart's boomerang          ┆ 0.710211 │
│ Sequoia                    ┆ 0.710122 │
│ Google and the W

In [37]:
# Titanic
related_movies("tt0120338")

shape: (100, 2)
┌────────────────────────┬──────────┐
│ title                  ┆ cossim   │
│ ---                    ┆ ---      │
│ str                    ┆ f64      │
╞════════════════════════╪══════════╡
│ Titanic                ┆ 1.0      │
│ Titanic                ┆ 0.86594  │
│ Raise the Titanic      ┆ 0.850919 │
│ Titanic Town           ┆ 0.848641 │
│ Reflections on Titanic ┆ 0.838914 │
│ S.O.S. Titanic         ┆ 0.832334 │
│ Titanic                ┆ 0.825235 │
│ Saving the Titanic     ┆ 0.823673 │
│ Titanic: 100 Years On  ┆ 0.819573 │
│ Titanic: 100 Years in  ┆ 0.816164 │
│ …                      ┆ …        │
│ Lucia                  ┆ 0.756986 │
│ Platinum               ┆ 0.75679  │
│ With or Without You    ┆ 0.756671 │
│ Hearts in Atlantis     ┆ 0.756415 │
│ Between Forever        ┆ 0.756322 │
│ Troubled Waters        ┆ 0.756251 │
│ Reg                    ┆ 0.756089 │
│ Dad                    ┆ 0.756085 │
│ Lilting                ┆ 0.756054 │
│ Helena                 ┆ 0.75600

In [38]:
# The Matrix
related_movies("tt0133093")

shape: (100, 2)
┌───────────────────────────────┬──────────┐
│ title                         ┆ cossim   │
│ ---                           ┆ ---      │
│ str                           ┆ f64      │
╞═══════════════════════════════╪══════════╡
│ The Matrix                    ┆ 1.0      │
│ The Matrix Reloaded           ┆ 0.954368 │
│ The Matrix Revolutions        ┆ 0.9392   │
│ The Matrix Resurrections      ┆ 0.901059 │
│ The Matrix: The Movie Special ┆ 0.878592 │
│ The Matrix: Reborn            ┆ 0.794133 │
│ Replicas                      ┆ 0.783146 │
│ Generation Um...              ┆ 0.764482 │
│ Hacked                        ┆ 0.763077 │
│ Guardian                      ┆ 0.761921 │
│ …                             ┆ …        │
│ Stay                          ┆ 0.741514 │
│ Lush                          ┆ 0.741408 │
│ Only                          ┆ 0.741335 │
│ The Contract                  ┆ 0.741328 │
│ Betrayal                      ┆ 0.741282 │
│ ...Almost                     ┆ 0.741

In [39]:
# Spirited Away
related_movies("tt0245429")

shape: (100, 2)
┌──────────────────────────────┬──────────┐
│ title                        ┆ cossim   │
│ ---                          ┆ ---      │
│ str                          ┆ f64      │
╞══════════════════════════════╪══════════╡
│ Spirited Away                ┆ 1.0      │
│ Spirited Away: Live on Stage ┆ 0.872152 │
│ Yuki                         ┆ 0.806406 │
│ Spirited                     ┆ 0.80101  │
│ Your Name.                   ┆ 0.790444 │
│ A Letter to Momo             ┆ 0.774361 │
│ Hal                          ┆ 0.773274 │
│ Family                       ┆ 0.77277  │
│ My Neighbor Totoro           ┆ 0.770817 │
│ Blame!                       ┆ 0.769622 │
│ …                            ┆ …        │
│ Hello World                  ┆ 0.741126 │
│ Liz and the Blue Bird        ┆ 0.741084 │
│ Ask This of Rikyu            ┆ 0.741036 │
│ Penguin Highway              ┆ 0.741025 │
│ Asylum Session               ┆ 0.740821 │
│ Nobuko                       ┆ 0.740819 │
│ Mukoku        

In [40]:
# Everything Everywhere All at Once
related_movies("tt6710474")

shape: (100, 2)
┌─────────────────────────────────┬──────────┐
│ title                           ┆ cossim   │
│ ---                             ┆ ---      │
│ str                             ┆ f64      │
╞═════════════════════════════════╪══════════╡
│ Everything Everywhere All at O… ┆ 1.0      │
│ Everything and Everyone         ┆ 0.823197 │
│ All at Once                     ┆ 0.814642 │
│ All at Once                     ┆ 0.809112 │
│ Everything at Once              ┆ 0.808358 │
│ Anywhere Somewhere Nowhere      ┆ 0.806519 │
│ Everywhere                      ┆ 0.79885  │
│ Love, Once and Always           ┆ 0.791778 │
│ All the Way                     ┆ 0.786891 │
│ Access All Areas                ┆ 0.786082 │
│ …                               ┆ …        │
│ Nothing to Lose                 ┆ 0.755258 │
│ Instant Issues                  ┆ 0.755165 │
│ Adventures in Success           ┆ 0.755096 │
│ Gunless                         ┆ 0.754681 │
│ Chronically Metropolitan        ┆ 0.754553

In [41]:
# The Avengers
related_movies("tt0848228")

shape: (100, 2)
┌─────────────────────────────────┬──────────┐
│ title                           ┆ cossim   │
│ ---                             ┆ ---      │
│ str                             ┆ f64      │
╞═════════════════════════════════╪══════════╡
│ The Avengers                    ┆ 1.0      │
│ The Avengers Assemble Premiere  ┆ 0.874893 │
│ The Avengers                    ┆ 0.856223 │
│ Avengers: Age of Ultron         ┆ 0.851094 │
│ Avengers: Infinity War          ┆ 0.851068 │
│ Avengers: Endgame               ┆ 0.84401  │
│ The Avengers                    ┆ 0.813166 │
│ Captain America: The First Ave… ┆ 0.798043 │
│ Alien Avengers                  ┆ 0.796697 │
│ The Avengers                    ┆ 0.782974 │
│ …                               ┆ …        │
│ Edge of Tomorrow                ┆ 0.715891 │
│ Iron Soldier                    ┆ 0.715866 │
│ 2020                            ┆ 0.715723 │
│ Avenged                         ┆ 0.715417 │
│ Space Cadet                     ┆ 0.715079